In [4]:
import numpy
import numpy as np
import time
import datetime
import netCDF4
import matplotlib.pylab as plt
import os,sys,glob
from multiprocessing import Pool
#import odb
from eccodes import *
from functools import partial
from collections import OrderedDict
import subprocess
import json
import gzip
import copy
import pickle
import xarray as xr
import pickle
import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
matplotlib.rcParams.update({'font.size': 20})

plt.rcParams['lines.linewidth'] = 3

import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua4 as eua
import trajectory as trj

import h5py
import ray
import pyproj
# ray.init(num_cpus=7)


In [5]:

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def datetime_to_seconds(dates, ref='1900-01-01T00:00:00'):
    """ from datetime64 to seconds since 1900-01-01 00:00:00"""
    return ((dates - np.datetime64(ref)) / np.timedelta64(1, 's')).astype(np.int64)

def seconds_to_datetime(seconds, ref='1900-01-01'):
    """ from seconds to datetime64 """
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit='s', origin=ref)

In [6]:
def get_cuon(stat, year_s, year_e):
    # stat = '11035'
    # year = '2000'
    dt_from = datetime_to_seconds(np.datetime64(str(year_s)+'-01-01'))
    dt_to = datetime_to_seconds(np.datetime64(str(year_e)+'-12-31'))

    conv_file = glob.glob('/mnt/users/scratch/leo/scratch/converted_v10/*' + stat + '*_CEUAS_merged_v1.nc')[0]
    # conv_file = glob.glob('./0-20001-0-11035_CEUAS_merged_v1.nc')[0]

    df_dict = {}
    u_df_dict = {}
    v_df_dict = {}

    stdplevs = [1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500]


    with h5py.File(conv_file, 'r') as file:
        rts = file['recordindices']['recordtimestamp'][:]
        idx = np.where(np.logical_and((rts >= dt_from), (rts <= dt_to)))[0]
        if len(idx) == 0:
            print('(1) NO DATA FOUND IN CONVERTED_V9: ', sid)
        u_idx = file['recordindices']['139'][idx]
        v_idx = file['recordindices']['140'][idx]
        t_idx = file['recordindices']['126'][idx]
        
#         plevs = [1000,2000,3000,5000,7000,10000,15000,20000,25000,30000,40000,50000,70000,85000,92500,100000]
        
#         mask = file['observations_table']['z_coordinate'][t_idx[0]:t_idx[-1]]
#         mask = np.isin(mask,plevs)

#         u_mask = file['observations_table']['z_coordinate'][u_idx[0]:u_idx[-1]]
#         u_mask = np.isin(u_mask,plevs)

#         v_mask = file['observations_table']['z_coordinate'][v_idx[0]:v_idx[-1]]
#         v_mask = np.isin(v_mask,plevs)
        mask = len(file['observations_table']['z_coordinate'][t_idx[0]:t_idx[-1]])*[True]
        u_mask = len(file['observations_table']['z_coordinate'][u_idx[0]:u_idx[-1]])*[True]
        v_mask = len(file['observations_table']['z_coordinate'][v_idx[0]:v_idx[-1]])*[True]

        # t_len = len(mask[mask == True])
        # u_len = len(u_mask[u_mask == True])
        # v_len = len(v_mask[v_mask == True])
        
        t_len = len(mask)
        u_len = len(u_mask)
        v_len = len(v_mask)

        df_dict['z_coordinate'] = list(file['observations_table']['z_coordinate'][t_idx[0]:t_idx[-1]][mask])
        df_dict['date_time'] = list(file['observations_table']['date_time'][t_idx[0]:t_idx[-1]][mask])
        df_dict['observation_value'] = list(file['observations_table']['observation_value'][t_idx[0]:t_idx[-1]][mask])
        df_dict['latitude'] = list(file['observations_table']['latitude'][t_idx[0]:t_idx[-1]][mask])
        df_dict['longitude'] = list(file['observations_table']['longitude'][t_idx[0]:t_idx[-1]][mask])
        repid = np.asarray(file['observations_table']['report_id'][t_idx[0]:t_idx[-1]][mask])
        df_dict['report_id'] = list(repid.view('|S{}'.format(repid.shape[1])).flatten().astype(str))

        # df_dict['RASE_bias_estimate'] = list(file['advanced_homogenisation']['RASE_bias_estimate'][t_idx[0]:t_idx[-1]][mask])
        df_dict['fg_depar'] = list(file['era5fb']['fg_depar@body'][t_idx[0]:t_idx[-1]][mask])
        df_dict['an_depar'] = list(file['era5fb']['an_depar@body'][t_idx[0]:t_idx[-1]][mask])
        df_dict['biascorr'] = list(file['era5fb']['biascorr@body'][t_idx[0]:t_idx[-1]][mask])
        # df_dict['latitude_displacement'] = list(file['advanced_homogenisation']['latitude_displacement'][t_idx[0]:t_idx[-1]][mask])
        # df_dict['longitude_displacement'] = list(file['advanced_homogenisation']['longitude_displacement'][t_idx[0]:t_idx[-1]][mask])
        # df_dict['time_since_launch'] = list(file['advanced_homogenisation']['time_since_launch'][t_idx[0]:t_idx[-1]][mask])

        u_df_dict['z_coordinate'] = list(file['observations_table']['z_coordinate'][u_idx[0]:u_idx[-1]][u_mask])
        u_df_dict['date_time'] = list(file['observations_table']['date_time'][u_idx[0]:u_idx[-1]][u_mask])
        u_df_dict['observation_value'] = list(file['observations_table']['observation_value'][u_idx[0]:u_idx[-1]][u_mask])

        v_df_dict['z_coordinate'] = list(file['observations_table']['z_coordinate'][v_idx[0]:v_idx[-1]][v_mask])
        v_df_dict['date_time'] = list(file['observations_table']['date_time'][v_idx[0]:v_idx[-1]][v_mask])
        v_df_dict['observation_value'] = list(file['observations_table']['observation_value'][v_idx[0]:v_idx[-1]][v_mask])


        # df_dict['observation_value'] = ['air_temperature']*t_len
        # u_df_dict['observation_value'] = ['eastward_wind']*u_len
        # v_df_dict['observation_value'] = ['northward_wind']*v_len

        df_dict['date_time'] = seconds_to_datetime(df_dict['date_time'])
        df = pd.DataFrame.from_dict(df_dict)
        df = df.rename(columns={'observation_value':'air_temperature'})

        u_df_dict['date_time'] = seconds_to_datetime(u_df_dict['date_time'])
        u_df = pd.DataFrame.from_dict(u_df_dict)
        u_df = u_df.rename(columns={'observation_value':'eastward_wind_speed'})

        v_df_dict['date_time'] = seconds_to_datetime(v_df_dict['date_time'])
        v_df = pd.DataFrame.from_dict(v_df_dict)
        v_df = v_df.rename(columns={'observation_value':'northward_wind_speed'})

        # put dfs together:
        df = df.merge(u_df, how='inner', on=['date_time','z_coordinate'])
        df = df.merge(v_df, how='inner', on=['date_time','z_coordinate'])
    #         print(df)
    return df

In [7]:
df = get_cuon('11035', '1980', '1993')

In [8]:
df

,z_coordinate,date_time,air_temperature,latitude,longitude,report_id,fg_depar,an_depar,biascorr,eastward_wind_speed,northward_wind_speed
0,3300.0,1980-01-01 00:00:00,205.699997,48.25,16.370001,100000000044957,2.663347,1.328656,-0.841882,27.574617,-4.862149
1,5000.0,1980-01-01 00:00:00,206.899994,48.25,16.370001,100000000044957,-0.888495,-0.555904,-0.068680,21.612930,-7.866463
2,5800.0,1980-01-01 00:00:00,206.300003,48.25,16.370001,100000000044957,-2.256762,-2.002759,-0.151446,17.726540,-3.125667
3,7000.0,1980-01-01 00:00:00,210.100006,48.25,16.370001,100000000044957,0.230430,0.207024,-0.256313,17.726540,-3.125667
4,10000.0,1980-01-01 00:00:00,212.500000,48.25,16.370001,100000000044957,-0.684438,-0.329867,-0.485842,16.741732,-2.952019
...,...,...,...,...,...,...,...,...,...,...,...
176368,85000.0,1993-12-30 12:00:00,272.500000,48.25,16.370001,100000000065624,-0.663564,-0.230999,0.307848,11.604740,-6.700000
176369,85900.0,1993-12-30 12:00:00,273.100006,48.25,16.370001,100000000065624,-0.484588,-0.174735,0.269503,11.604740,-6.700000
176370,92500.0,1993-12-30 12:00:00,276.000000,48.25,16.370001,100000000065624,0.112498,0.411676,0.000000,13.909332,-3.726994
176371,97200.0,1993-12-30 12:00:00,270.500000,48.25,16.370001,100000000065624,-5.890202,-4.865841,0.000000,7.705480,2.804565


In [14]:
conv_file = '/mnt/users/scratch/leo/scratch/converted_v9/newindex/0-20001-0-11035_CEUAS_merged_v1.nc'
#conv_file = '/mnt/users/scratch/leo/scratch/converted_v10/0-20001-0-11035_CEUAS_merged_v1.nc'

results = {}
results['group'] = []
results['variable'] = []
results['len'] = []
results['allnan'] = []
results['allequ'] = []
results['value_if_all_same'] = []


with h5py.File(conv_file, 'r') as file:
    print(file['station_configuration'])
    print(file['station_configuration'].keys())
    print(file['station_configuration']['primary_id'][:])
    print(np.unique(file['station_configuration']['primary_id'][:]))
    
#     for i in file.keys():
#         print(i)
#         print('    ',file[i])
#         do_vars = True
#         try:
#             for j in file[i].keys():
#                 x = 0
#         except:
#             do_vars = False
#         if do_vars:
#             for j in file[i].keys():
#                 print('        ',j)
#                 print('            ',file[i][j])
#                 print('            ',file[i][j][-10:])
#                 try:
#                     allnan = np.all(np.isnan(file[i][j][:]))
#                 except:
#                     allnan = False
#                 print('            ', allnan)
#                 allequ = np.all(file[i][j][:] == file[i][j][-1])
#                 print('            ', allequ)
                
#                 if allnan:
#                     results['value_if_all_same'].append('nan')
#                 elif allequ:
#                     results['value_if_all_same'].append(str(file[i][j][-1]))
#                 else:
#                     results['value_if_all_same'].append('')
#                 results['len'].append(np.shape(file[i][j][:]))
#                 results['group'].append(i)
#                 results['variable'].append(j)
#                 results['allnan'].append(allnan)
#                 results['allequ'].append(allequ)
#         print()
# rdf_old = pd.DataFrame.from_dict(results)
# display(rdf)
# display(rdf_old)

<HDF5 group "/station_configuration" (46 members)>
<KeysViewHDF5 ['alternative_name', 'bbox_max_latitude', 'bbox_max_longitude', 'bbox_min_latitude', 'bbox_min_longitude', 'city', 'comment', 'contact', 'end_date', 'index', 'latitude', 'local_gravity', 'longitude', 'measuring_system_id', 'measuring_system_model', 'metadata_contact', 'metadata_contact_role', 'observed_variables', 'observing_frequency', 'operating_institute', 'operating_territory', 'optional_data', 'platform_sub_type', 'platform_type', 'primary_id', 'primary_id_scheme', 'reporting_time', 'role', 'secondary_id', 'secondary_id_scheme', 'start_date', 'station_abbreviation', 'station_automation', 'station_crs', 'station_name', 'station_type', 'string12', 'string15', 'string21', 'string22', 'string25', 'string3', 'string32', 'string34', 'string83', 'telecommunication_method']>
[[b'0' b'-' b'2' b'0' b'0' b'0' b'1' b'-' b'0' b'-' b'1' b'1' b'0' b'3'
  b'5']]
[b'-' b'0' b'1' b'2' b'3' b'5']


In [24]:
rdf_old.to_csv('station_check_old.csv')

In [39]:
rdf_old.

,group,variable,len,allnan,allequ,value_if_all_same
0,advanced_homogenisation,RAOBCORE_bias_estimate,"(88072698,)",False,False,
1,advanced_homogenisation,RASE_bias_estimate,"(88072698,)",False,False,
2,advanced_homogenisation,RICH_bias_estimate,"(88072698,)",False,False,
3,advanced_homogenisation,RISE_bias_estimate,"(88072698,)",False,False,
4,advanced_homogenisation,humidity_bias_estimate,"(88072698,)",False,False,
...,...,...,...,...,...,...
300,units,units_len,"(154,)",False,True,0.0
301,z_coordinate_type,description,"(2, 80)",False,False,
302,z_coordinate_type,string80,"(80,)",False,True,0.0
303,z_coordinate_type,type,"(2,)",False,False,


In [46]:
rdf['path'] = np.array(rdf.group +'/'+ rdf.variable)

In [47]:
rdf_old['path'] = np.array(rdf_old.group +'/'+ rdf_old.variable)

In [56]:
rdf_comp = rdf[np.isin(rdf.path, rdf_old.path)]
rdf_comp['old_value_if_all_same'] = np.array(rdf_old[np.isin(rdf_old.path, rdf.path)].value_if_all_same)
rdf_comp


,group,variable,len,allnan,allequ,value_if_all_same,path,old_value_if_all_same
0,crs,crs,"(1,)",False,True,0,crs/crs,0
1,crs,crs_len,"(1,)",False,True,0.0,crs/crs_len,0.0
2,crs,description,"(1, 80)",False,True,[b'w' b'g' b's' b'8' b'4' b'' b'' b'' b'' b'' ...,crs/description,[b'w' b'g' b's' b'8' b'4' b'' b'' b'' b'' b'' ...
3,crs,string80,"(80,)",False,True,0.0,crs/string80,0.0
4,era5fb,albedo@modsurf,"(115828893,)",False,False,,era5fb/albedo@modsurf,
...,...,...,...,...,...,...,...,...
282,units,units_len,"(154,)",False,True,0.0,units/units_len,0.0
283,z_coordinate_type,description,"(2, 80)",False,False,,z_coordinate_type/description,
284,z_coordinate_type,string80,"(80,)",False,True,0.0,z_coordinate_type/string80,0.0
285,z_coordinate_type,type,"(2,)",False,False,,z_coordinate_type/type,


In [60]:
test_df = rdf_comp[rdf_comp.value_if_all_same != rdf_comp.old_value_if_all_same]

In [63]:
test_df.loc[249].old_value_if_all_same

"[b'1' b'9' b'1' b'0' b'0' b'1' b'0' b'6' b'' b'' b'' b'' b'' b'' b'' b''\n b'' b'' b'' b'' b'']"

In [57]:
rdf_comp.to_csv('comparison_csv_nc_check.csv')

In [64]:
rdf[~np.isin(rdf.path, rdf_old.path)]

,group,variable,len,allnan,allequ,value_if_all_same,path
127,header_table,string21,"(20,)",False,True,b'',header_table/string21
174,observations_table,string21,"(20,)",False,True,b'',observations_table/string21
255,station_configuration,string1,"(1,)",False,True,b'',station_configuration/string1
262,station_configuration,string4,"(4,)",False,True,b'',station_configuration/string4
263,station_configuration,string8,"(8,)",False,True,b'',station_configuration/string8


In [65]:
rdf_old[~np.isin(rdf_old.path, rdf.path)]

,group,variable,len,allnan,allequ,value_if_all_same,path
0,advanced_homogenisation,RAOBCORE_bias_estimate,"(88072698,)",False,False,,advanced_homogenisation/RAOBCORE_bias_estimate
1,advanced_homogenisation,RASE_bias_estimate,"(88072698,)",False,False,,advanced_homogenisation/RASE_bias_estimate
2,advanced_homogenisation,RICH_bias_estimate,"(88072698,)",False,False,,advanced_homogenisation/RICH_bias_estimate
3,advanced_homogenisation,RISE_bias_estimate,"(88072698,)",False,False,,advanced_homogenisation/RISE_bias_estimate
4,advanced_homogenisation,humidity_bias_estimate,"(88072698,)",False,False,,advanced_homogenisation/humidity_bias_estimate
5,advanced_homogenisation,index,"(88072698,)",False,True,b'',advanced_homogenisation/index
6,advanced_homogenisation,latitude_displacement,"(88072698,)",False,False,,advanced_homogenisation/latitude_displacement
7,advanced_homogenisation,longitude_displacement,"(88072698,)",False,False,,advanced_homogenisation/longitude_displacement
8,advanced_homogenisation,time_since_launch,"(88072698,)",False,False,,advanced_homogenisation/time_since_launch
9,advanced_homogenisation,true_time,"(88072698,)",False,False,,advanced_homogenisation/true_time


In [50]:
rdf[np.isin(rdf.path, rdf_old.path)]

,group,variable,len,allnan,allequ,value_if_all_same,path
0,crs,crs,"(1,)",False,True,0,crs/crs
1,crs,crs_len,"(1,)",False,True,0.0,crs/crs_len
2,crs,description,"(1, 80)",False,True,[b'w' b'g' b's' b'8' b'4' b'' b'' b'' b'' b'' ...,crs/description
3,crs,string80,"(80,)",False,True,0.0,crs/string80
4,era5fb,albedo@modsurf,"(115828893,)",False,False,,era5fb/albedo@modsurf
...,...,...,...,...,...,...,...
282,units,units_len,"(154,)",False,True,0.0,units/units_len
283,z_coordinate_type,description,"(2, 80)",False,False,,z_coordinate_type/description
284,z_coordinate_type,string80,"(80,)",False,True,0.0,z_coordinate_type/string80
285,z_coordinate_type,type,"(2,)",False,False,,z_coordinate_type/type


In [38]:
add_col = []
for i in rdf_old.group.drop_duplicates():
    #print(i)
    irdf = rdf[rdf.group == i]
    for j in irdf.variable:
        virdf = irdf[irdf.variable == j]
        if len(rdf[np.logical_and(rdf.group == i, rdf.variable ==j)]) > 0:
            display(rdf[np.logical_and(rdf.group == i, rdf.variable ==j)])
            print(i,j)
        

,group,variable,len,allnan,allequ,value_if_all_same
0,crs,crs,"(1,)",False,True,0


crs crs


,group,variable,len,allnan,allequ,value_if_all_same
1,crs,crs_len,"(1,)",False,True,0.0


crs crs_len


,group,variable,len,allnan,allequ,value_if_all_same
2,crs,description,"(1, 80)",False,True,[b'w' b'g' b's' b'8' b'4' b'' b'' b'' b'' b'' ...


...

z_coordinate_type string80


,group,variable,len,allnan,allequ,value_if_all_same
285,z_coordinate_type,type,"(2,)",False,False,


z_coordinate_type type


,group,variable,len,allnan,allequ,value_if_all_same
286,z_coordinate_type,z_coordinate_type_len,"(2,)",False,True,0.0


z_coordinate_type z_coordinate_type_len


In [26]:
rdf

,group,variable,len,allnan,allequ,value_if_all_same
0,crs,crs,"(1,)",False,True,0
1,crs,crs_len,"(1,)",False,True,0.0
2,crs,description,"(1, 80)",False,True,[b'w' b'g' b's' b'8' b'4' b'' b'' b'' b'' b'' ...
3,crs,string80,"(80,)",False,True,0.0
4,era5fb,albedo@modsurf,"(115828893,)",False,False,
...,...,...,...,...,...,...
282,units,units_len,"(154,)",False,True,0.0
283,z_coordinate_type,description,"(2, 80)",False,False,
284,z_coordinate_type,string80,"(80,)",False,True,0.0
285,z_coordinate_type,type,"(2,)",False,False,


In [6]:
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua4 as eua

In [7]:
def request(rqdict):
    data = eua.vm_request_wrapper(rqdict, overwrite=True, vm_url='http://srvx1.img.univie.ac.at:8007')
    
    return data

In [31]:
var = ['air_temperature', 'relative_humidity', 'eastward_wind_speed']
opt = ''
for i in var:
    rd={
        'variable': i,
        'statid': '11035',
        # 'optional': opt,
        'date': '20191230-20191231',
    } 
    display(request(rd).to_dataframe())

,lat,lon,obs,plev,report_id,station_id,ta,time
0,48.248951,16.356859,0,560.0,100000000116164,WIEN/HOHE WARTE (11035-1),227.250000,2019-12-30 11:32:22
1,48.248951,16.356859,0,570.0,100000000116164,WIEN/HOHE WARTE (11035-1),227.559998,2019-12-30 11:32:22
2,48.248951,16.356859,0,580.0,100000000116164,WIEN/HOHE WARTE (11035-1),227.610001,2019-12-30 11:32:22
3,48.248951,16.356859,0,590.0,100000000116164,WIEN/HOHE WARTE (11035-1),227.710007,2019-12-30 11:32:22
4,48.248951,16.356859,0,600.0,100000000116164,WIEN/HOHE WARTE (11035-1),227.289993,2019-12-30 11:32:22
...,...,...,...,...,...,...,...,...
5667,48.248951,16.357180,0,93430.0,100000000116167,WIEN/HOHE WARTE (11035-1),272.959991,2019-12-31 23:41:21
5668,48.248951,16.357180,0,97960.0,100000000116167,WIEN/HOHE WARTE (11035-1),275.779999,2019-12-31 23:41:21
5669,48.248951,16.357180,0,100000.0,100000000116167,WIEN/HOHE WARTE (11035-1),277.309998,2019-12-31 23:41:21
5670,48.248951,16.357180,0,100440.0,100000000116167,WIEN/HOHE WARTE (11035-1),277.570007,2019-12-31 23:41:21


,hur,lat,lon,obs,plev,report_id,station_id,time
0,0.010622,48.248951,16.356859,0,560.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22
1,0.010620,48.248951,16.356859,0,570.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22
2,0.010630,48.248951,16.356859,0,580.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22
3,0.010616,48.248951,16.356859,0,590.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22
4,0.010610,48.248951,16.356859,0,600.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22
...,...,...,...,...,...,...,...,...
5666,0.761943,48.248951,16.357180,0,93430.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21
5667,0.679380,48.248951,16.357180,0,97960.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21
5668,0.621206,48.248951,16.357180,0,100000.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21
5669,0.616800,48.248951,16.357180,0,100440.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21


,lat,lon,obs,plev,report_id,station_id,time,ua
0,48.248951,16.356859,0,560.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22,72.345924
1,48.248951,16.356859,0,570.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22,76.125641
2,48.248951,16.356859,0,580.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22,75.094482
3,48.248951,16.356859,0,590.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22,72.675148
4,48.248951,16.356859,0,600.0,100000000116164,WIEN/HOHE WARTE (11035-1),2019-12-30 11:32:22,72.481094
...,...,...,...,...,...,...,...,...
5667,48.248951,16.357180,0,93430.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21,6.128872
5668,48.248951,16.357180,0,97960.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21,11.894828
5669,48.248951,16.357180,0,100000.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21,6.920714
5670,48.248951,16.357180,0,100440.0,100000000116167,WIEN/HOHE WARTE (11035-1),2019-12-31 23:41:21,4.886402


In [8]:
var = ['air_temperature']
opt = ''
for i in var:
    rd={
        'variable': i,
        'statid': '11035',
        # 'optional': opt,
        # 'date': '20191230-20191231',
    } 
    display(request(rd).to_dataframe())

,lat,lon,obs,plev,report_id,station_id,ta,time
0,48.250000,16.370001,0,4300.0,800000000000871,WIEN/HOHE WARTE (11035-1),220.750000,1925-01-24 09:22:00
1,48.250000,16.370001,0,4400.0,800000000000871,WIEN/HOHE WARTE (11035-1),220.550003,1925-01-24 09:22:00
2,48.250000,16.370001,0,5300.0,800000000000871,WIEN/HOHE WARTE (11035-1),216.250000,1925-01-24 09:22:00
3,48.250000,16.370001,0,9200.0,800000000000871,WIEN/HOHE WARTE (11035-1),215.850006,1925-01-24 09:22:00
4,48.250000,16.370001,0,10300.0,800000000000871,WIEN/HOHE WARTE (11035-1),220.050003,1925-01-24 09:22:00
...,...,...,...,...,...,...,...,...
12372040,48.248871,16.357590,0,99440.0,100000000118586,WIEN/HOHE WARTE (11035-1),279.660004,2022-12-31 11:30:23
12372041,48.248871,16.357590,0,99470.0,100000000118586,WIEN/HOHE WARTE (11035-1),279.799988,2022-12-31 11:30:23
12372042,48.248871,16.357590,0,99530.0,100000000118586,WIEN/HOHE WARTE (11035-1),279.920013,2022-12-31 11:30:23
12372043,48.248871,16.357590,0,99540.0,100000000118586,WIEN/HOHE WARTE (11035-1),280.049988,2022-12-31 11:30:23


In [9]:
rd={
    'variable': 'air_temperature',
    'statid': '11035',
    'year': '2010',
    'month': '01',
    'day':'01',
} 
display(request(rd).to_dataframe())

,lat,lon,obs,plev,report_id,station_id,ta,time
0,48.25,16.370001,0,640.0,100000000096234,WIEN/HOHE WARTE (11035-1),224.500000,2010-01-01 00:00:00
1,48.25,16.370001,0,700.0,100000000096234,WIEN/HOHE WARTE (11035-1),223.300003,2010-01-01 00:00:00
2,48.25,16.370001,0,790.0,100000000096234,WIEN/HOHE WARTE (11035-1),224.500000,2010-01-01 00:00:00
3,48.25,16.370001,0,880.0,100000000096234,WIEN/HOHE WARTE (11035-1),220.500000,2010-01-01 00:00:00
4,48.25,16.370001,0,1000.0,100000000096234,WIEN/HOHE WARTE (11035-1),224.300003,2010-01-01 00:00:00
...,...,...,...,...,...,...,...,...
115,48.25,16.370001,0,94600.0,100000000096237,WIEN/HOHE WARTE (11035-1),279.799988,2010-01-01 12:00:00
116,48.25,16.370001,0,95300.0,100000000096237,WIEN/HOHE WARTE (11035-1),279.799988,2010-01-01 12:00:00
117,48.25,16.370001,0,95400.0,100000000096237,WIEN/HOHE WARTE (11035-1),279.000000,2010-01-01 12:00:00
118,48.25,16.370001,0,95800.0,100000000096237,WIEN/HOHE WARTE (11035-1),278.399994,2010-01-01 12:00:00
